In [ ]:
import matplotlib as mpl
from matplotlib import pyplot as plt
from keras.optimizers import Adam
from pretrained_seg_models import Unet, AttentionUnet, AttentionResUnet, get_preprocessing, losses, metrics
from mpl_toolkits import mplot3d
from keras.models import load_model
import pandas as pd
from statistical_analysis.maths_stats import get_class_volume_msd, get_CIs

In [ ]:
# Define paths for dateset and the number of classes in the dataset

HPF = 48
PATH = '/content/drive/MyDrive/Data/Train{}/'.format(HPF)
TEST_PATH = '/content/drive/MyDrive/Data/Test{}/'.format(HPF)
OUT_PATH = '/content/drive/MyDrive/Data/Results'
HEIGHT = 256
WIDTH = 256
DEPTH = 256
CHANNELS = 3
if HPF == 48:
    n_classes = 6
elif HPF == 36:
    n_classes = 5
elif HPF == 30:
    n_classes = 4
else:
    n_classes = 1

In [ ]:
# Define model parameters

encoder_weights = 'imagenet'
BACKBONE1 = 'resnet34'
activation = 'softmax'
patch_size = 64
channels = 3

LR = 0.0001
opt = Adam(LR)

flat_train_masks = train_masks.reshape(-1)
class_weights = compute_class_weight('balanced', classes=np.unique(flat_train_masks), y=flat_train_masks)

DL = losses.DiceLoss(class_weights=class_weights)
FL = losses.CategoricalFocalLoss()
total_loss = DL + (1 * FL)

metrics = [metrics.IOUScore(threshold=0.5), metrics.FScore(threshold=0.5)]

In [ ]:
# Preprocess input data

preprocess_input1 = get_preprocessing(BACKBONE1)
x_train_prep = preprocess_input1(x_train)
x_val_prep = preprocess_input1(x_val)

In [ ]:
# Define model - using AttentionResUnet with a resnet34 backbone and 
# pretrained weights

model1 = AttentionResUnet(BACKBONE1, classes=n_classes, 
                            input_shape=(patch_size, patch_size, patch_size, channels), 
                            encoder_weights=encoder_weights, activation=activation)
model1.compile(optimizer=opt, loss=total_loss, metrics=metrics)
model1.summary()

In [ ]:
# Train the model

history1 = model1.fit(x_train_prep, y_train, batch_size=8, epochs=100, verbose=1,
                    validation_data=(x_val_prep, y_val))

In [ ]:
# Save the model for use in the future

model1.save(OUT_PATH+'{}HPF_{}_atten_resunet_100epochs.h5'.format(HPF, BACKBONE1))

In [ ]:
# Plot training and validation loss and accuracy at each epoch

loss = history1.history['loss']
val_loss = history1.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and Validation Loss for AttentionResUNet with ResNet34 backbone')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = history1.history['iou_score']
val_acc = history1.history['val_iou_score']
plt.plot(epochs, acc, 'y', label='Training IOU')
plt.plot(epochs, val_acc, 'r', label='Validation IOU ')
plt.title('Training and Validation IOU for AttentionResUNet with ResNet34 backbone')
plt.xlabel('Epochs')
plt.ylabel('IOU')
plt.legend()
plt.show()

In [ ]:
# Preprocess input data with vgg16 backbone

BACKBONE2 = 'vgg16'

preprocess_input2 = get_preprocessing(BACKBONE2)
x_train_prep = preprocess_input2(x_train)
x_val_prep = preprocess_input2(x_val)

In [ ]:
# Define model - using AttentionUnet with a vgg16 backbone and 
# pretrained weights

model2 = AttentionUnet(BACKBONE2, classes=n_classes, 
                            input_shape=(patch_size, patch_size, patch_size, channels), 
                            encoder_weights=encoder_weights, activation=activation)
model2.compile(optimizer=opt, loss=total_loss, metrics=metrics)
model2.summary()

In [ ]:
# Train the model

history2 = model2.fit(x_train_prep, y_train, batch_size=8, epochs=100, verbose=1,
                    validation_data=(x_val_prep, y_val))

In [ ]:
# Plot training and validation loss and accuracy at each epoch

loss = history2.history['loss']
val_loss = history2.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and Validation Loss for AttentionUNet with VGG16 backbone')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = history2.history['iou_score']
val_acc = history2.history['val_iou_score']
plt.plot(epochs, acc, 'y', label='Training IOU')
plt.plot(epochs, val_acc, 'r', label='Validation IOU')
plt.title('Training and Validation IOU for AttentionUNet with VGG16 backbone')
plt.xlabel('Epochs')
plt.ylabel('IOU')
plt.legend()
plt.show()

In [ ]:
# Define model - using Unet with a vgg16 backbone and 
# pretrained weights

model3 = Unet(BACKBONE2, classes=n_classes, 
                            input_shape=(patch_size, patch_size, patch_size, channels), 
                            encoder_weights=encoder_weights, activation=activation)
model3.compile(optimizer=opt, loss=total_loss, metrics=metrics)
model3.summary()

In [ ]:
# Train the model

history3 = model3.fit(x_train_prep, y_train, batch_size=8, epochs=100, verbose=1,
                    validation_data=(x_val_prep, y_val))

In [ ]:
# Plot training and validation loss and accuracy at each epoch

loss = history3.history['loss']
val_loss = history3.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and Validation Loss for UNet with VGG16 backbone')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = history3.history['iou_score']
val_acc = history3.history['val_iou_score']
plt.plot(epochs, acc, 'y', label='Training IOU')
plt.plot(epochs, val_acc, 'r', label='Validation IOU')
plt.title('Training and Validation IOU for UNet with VGG16 backbone')
plt.xlabel('Epochs')
plt.ylabel('IOU')
plt.legend()
plt.show()

In [ ]:
# Plot the actual and predicted masks for each patch in the validation set

fig = plt.figure(figsize=(8, 4*len(y_val)))

for i in range(len(y_val)):
    ax = fig.add_subplot(len(y_val), 2, (2*i)+1, projection='3d')
    ax.set_title('Actual Mask')
    for j in range(n_classes):
        if j != 0:
            points = np.nonzero(y_val[i,:,:,:,0] == 0.2*j)
            y = points[0]
            x = points[1]
            z = points[2]
            c1 = np.full(len(y), '{}'.format(0.2*j), dtype=np.float32)
            if y.shape[0] != 0:
                ax.scatter(x, y, z, c=c1, cmap='gray', alpha=1)

    ax = fig.add_subplot(len(y_val), 2, (2*i)+2, projection='3d')
    ax.set_title('Predicted Mask')
    for j in range(n_classes):
        if j != 0:
            points = np.nonzero(val_preds[i,:,:,:,0] == 0.2*j)
            y = points[0]
            x = points[1]
            z = points[2]
            c2 = np.full(len(y), '{}'.format(0.2*j), dtype=np.float32)
            if y.shape[0] != 0:
                ax.scatter(x, y, z, c=c2, cmap='gray', alpha=1)
plt.show()

In [ ]:
# Load the model for testing and predictions

my_model = load_model(OUT_PATH+'{}HPF_resnet34_atten_resunet_100epochs.h5'.format(HPF), compile=False)

In [ ]:
from tifffile import imsave

# Save masks as segmented volumes

for i in reconstructed_imgs:
    imsave(OUT_PATH+'fish{}_{}HPF.tif'.format(i, HPF), reconstructed_imgs[i])

In [ ]:
# Calculate mean and standard deviation of the volume of each area of the heart
# for a healthy fish (no GM) at each stage of development

if HPF == 48:
    classes = ['Background', 'Noise', 'Endocardium', 'Atrium', 'AVC', 'Ventricle']
    train_masks = np.expand_dims(, axis=4)
    healthy_masks = np.concatenate((train_masks, reconstructed_imgs), axis=0)
    healthy_scales = [295.53, 233.31, 233.31, 246.27, 246.27]
elif HPF == 36:
    classes = ['Background', 'Noise', 'Endocardium', 'Atrium', 'Ventricle']
    train_masks = np.expand_dims(masks, axis=4)
    healthy_masks = np.concatenate((train_masks, reconstructed_imgs), axis=0)
    healthy_scales = [221.65, 221.65, 221.65, 221.65, 221.65, 221.65]
elif HPF == 30:
    classes = ['Background', 'Noise', 'Endocardium', 'Linear Heart Tube']
    train_masks = np.expand_dims(masks, axis=4)
    healthy_masks = np.concatenate((train_masks, reconstructed_imgs), axis=0)
    healthy_scales = [221.65, 221.65]

msd_class_vols_df = get_class_volume_msd(healthy_masks, classes, healthy_scales, '{}'.format(HPF), 'Healthy')

In [ ]:
# Use the means and standard deviations of each class to calculate the
# confidence intervals of the healthy volumes

sds = msd_class_vols_df['Standard Deviation of Healthy Volume (\u03bcm\u00b2) at {}HPF'.format(HPF)].tolist()
class_vol_CIs_df = get_CIs(sds, len(healthy_masks), classes, '{}'.format(HPF), 'Healthy')

In [ ]:
# Remove background, noise and endocardium rows as these aren't of interest and were only included
# so the model produces better predictions of the other classes

msd_class_vols_df = msd_class_vols_df.drop(['Background', 'Noise', 'Endocardium'])
class_vol_CIs_df = class_vol_CIs_df.drop(['Background', 'Noise', 'Endocardium'])

In [ ]:
# Add all information into one dataframe, display it and save it as a CSV file
# which can be viewed outside of the code or terminal

stats = pd.DataFrame()
stats = stats.append(msd_class_vols_df)
df = stats.append(class_vol_CIs_df)

print(stats)

stats.to_csv(OUT_PATH+'{}HPF_stat_results.csv'.format(HPF))